In [1]:
from src.dl.inference.inferer import Inferer
import src.dl.lightning as lightning
from src.config import CONFIG

In [2]:
lightning_model = lightning.SegModel.from_conf(CONFIG)

inferer = Inferer(
    lightning_model,
    dataset="consep",
    stride_size=80,
    fn_pattern="*",
    model_weights="last",
    apply_weights=False,
    # post_proc_method="hover"
)

In [3]:
inferer.run_inference()

Inference: /home/local/leos/Dippa/datasets/data/consep/test/images: 100%|██████████| 2/2 [01:04<00:00, 32.06s/batch, patches=4046/4046]


In [4]:
inferer.post_process()

100%|██████████| 14/14 [00:56<00:00,  4.06s/it]


In [5]:
# import matplotlib.pyplot as plt
# from skimage.color import label2rgb

# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# ax.imshow(label2rgb(inferer.inst_maps["test_11"], bg_label=0))

In [6]:
pattern_list = None
binary_scores = inferer.benchmark_insts(pattern_list=pattern_list, file_prefix="2")
binary_scores

100%|██████████| 14/14 [01:40<00:00,  7.15s/it]


,AJI,AJI_plus,DICE2,PQ,SQ,DQ,inst_recall,inst_precision,splits,merges
name,,,,,,,,,,
test_1,0.472926,0.496966,0.407889,0.396094,0.726172,0.545455,0.600000,0.500000,0.645824,0.670128
test_10,0.545739,0.555669,0.453285,0.504908,0.758918,0.665300,0.768047,0.586799,0.883562,0.445246
test_11,0.415675,0.459884,0.401134,0.310895,0.701837,0.442974,0.403900,0.490417,0.770737,0.823053
test_12,0.559337,0.573326,0.538440,0.496633,0.751956,0.660455,0.745327,0.592937,0.516041,0.334265
test_13,0.547408,0.578215,0.599701,0.501528,0.757609,0.661988,0.624724,0.703980,0.689242,0.873267
test_14,0.562553,0.564529,0.564365,0.499689,0.769707,0.649194,0.868705,0.518240,1.227664,0.301579
test_2,0.440814,0.444751,0.536686,0.317227,0.701292,0.452347,0.648980,0.347162,0.545372,0.378709
test_3,0.413800,0.421724,0.359834,0.302273,0.726258,0.416206,0.573604,0.326590,0.668719,0.585533
test_4,0.436491,0.444452,0.374085,0.321366,0.722144,0.445017,0.619617,0.347185,0.837691,0.565123


In [7]:
pattern_list = None
type_scores = inferer.benchmark_types(pattern_list=pattern_list, file_prefix="1")
type_scores = type_scores[type_scores.index.str.contains("avg")]
type_scores

 43%|████▎     | 6/14 [00:01<00:01,  4.39it/s]